In [ ]:
import pandas as pd
import numpy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
jobs = pd.read_csv('dataset.csv')

In [ ]:
jobs.head(5)

In [ ]:
jobs.info()

In [ ]:
jobs = jobs[['Job Title','Job Salary','Job Experience Required','Key Skills','Location','Industry','Role']]
jobs.shape

In [ ]:
jobs.dropna(inplace=True)

In [ ]:
jobs.isnull().sum()

In [194]:
jobs['Key Skills'].unique()

array(['media planning digital media',
       'pre sales closing software knowledge clients requirements negotiating client online bidding good communication technology',
       'computer science fabrication quality check intellectual property electronics support services research management human resource management research executive',
       ...,
       'service delivery lms cre integration services content development client management crm',
       'security analysis software development life cycle security testing cyber security application security new projects code review project development software solutions security services',
       'press release public relations publishing web technologies seo writing journalism ms word syndication business executive search engine optimization'],
      dtype=object)

In [195]:
jobs['Location'].unique()

array(['Mumbai', 'Pune,Pune', 'Gurgaon', ...,
       'Delhi NCR (New Friends Colony) ', 'Bengaluru,Bhubaneshwar',
       'Howrah'], dtype=object)

In [ ]:
jobs.columns

In [ ]:
jobs['Key Skills'] = jobs['Key Skills'].apply(lambda x: x.lower().replace("|"," ").replace("  "," ").strip())
jobs['Key Skills'].unique()[5]
jobs['Tags'] = jobs['Key Skills'] +  ' ' + jobs['Location'] + ' ' + jobs['Role']
jobs['Tags'] = jobs['Tags'].apply(lambda x: x.lower().replace("  "," ").strip())

In [ ]:
salaryVlocation = jobs.groupby(jobs['Job Salary'])['Location'].sum().sort_values()
salaryVlocation = pd.DataFrame(salaryVlocation)
salaryVlocation.head(15)

In [ ]:
salaryVlocation = jobs.groupby(jobs['Key Skills'])['Location'].sum().sort_values()
salaryVlocation = pd.DataFrame(salaryVlocation)
salaryVlocation.head(15)

In [ ]:
salaryVlocation = jobs.groupby(jobs['Key Skills'])['Job Salary'].sum().sort_values()
salaryVlocation = pd.DataFrame(salaryVlocation)
salaryVlocation.head(15)

In [ ]:
salaryVlocation = jobs.groupby(jobs['Job Salary'])['Location'].sum().sort_values()
salaryVlocation = pd.DataFrame(salaryVlocation)
salaryVlocation.head(15)

In [ ]:
cv = CountVectorizer(max_features=1000,stop_words='english')

In [ ]:
def vectorize(narr):
    vector = cv.fit_transform(narr)
    similarity = cosine_similarity(vector)
    return similarity

In [ ]:
def getUserData():
    skills = input("Enter Your Skills (Mandatory) (ex: C, C++, Python)")
    experience = int(input("Enter Your Experience (ex: 0,1,2..)"))
    role = input("Enter If There are any Preffered Role (Optional)")
    location = input("Enter any Preffered Location (Optional)")
    userData = {'skills':skills, 'experience':experience, 'role':role, 'location':location }
    return userData

In [ ]:
def recommendJobs():
    userData = getUserData()
    tags = userData['skills'] + ' ' + userData['location'] + ' ' + userData['role']
    temp = {
        'Job Title':' ',
        'Job Salary':" ",
        'Job Experience Required': userData['experience'] or ' ',
        'Key Skills': userData['skills'] or ' ',
        'Location':userData['location'] or ' ',
        'Industry':' ',
        'Role':userData['role'] or ' ',
        'Tags': tags.lower()
    }
    newDf = jobs.append(temp,ignore_index = True)
    tempArray = newDf['Tags'].values
    vector = vectorize(tempArray)
    recommended_list = sorted(list(enumerate(vector[-1])),reverse=True,key= lambda x:x[1])
    recList = []
    for i in recommended_list[1:10]:
        recList.append(jobs.iloc[i[0]])
    recommendedJobs = pd.DataFrame(recList)
    recommendedJobs = recommendedJobs[['Job Title','Job Salary','Job Experience Required', 'Key Skills','Location','Industry','Role']]
    return recommendedJobs

In [ ]:
recommendedJobs = recommendJobs()
recommendedJobs

In [ ]:
salaryVlocation = recommendedJobs.groupby(jobs['Job Salary'])['Location'].sum().sort_values()
salaryVlocation = pd.DataFrame(salaryVlocation)
salaryVlocation.head(10)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
figure_name = plt.figure(figsize=(200, 100))
fruits = salaryVlocation.index
counts = salaryVlocation['Location']

ax.bar( counts,fruits, label=bar_labels, color=bar_colors)

plt.show()

In [ ]:
roleVindustry = recommendedJobs.groupby(jobs['Role'])['Industry'].sum().sort_values()
roleVindustry = pd.DataFrame(roleVindustry)
roleVindustry.head(10)

In [ ]:
skillsVlocation = recommendedJobs.groupby(jobs['Key Skills'])['Location'].sum().sort_values()
skillsVlocation = pd.DataFrame(skillsVlocation)
skillsVlocation.head(15)

In [ ]:
fig, ax = plt.subplots()
figure_name = plt.figure(figsize=(200, 100))
fruits = skillsVlocation.index
counts = skillsVlocation['Location']

ax.bar( counts,fruits, label=bar_labels, color=bar_colors)


plt.show()